In [1]:
from time import time
from tqdm import tqdm
from loguru import logger
import sys

In [2]:
raw_data = open('files/input_day5.txt').read()

In [3]:
example = """3-5
10-14
16-20
12-18

1
5
8
11
17
32"""
example

'3-5\n10-14\n16-20\n12-18\n\n1\n5\n8\n11\n17\n32'

In [4]:
def preprocess(data):
    fresh_ingerdient_ranges = [[int(rng.split('-')[0]), int(rng.split('-')[1])] for rng in data.split("\n\n")[0].split("\n")]
    fresh_ingerdient_ranges.sort()
    available_ingredients = [int(ing) for ing in data.strip().split("\n\n")[1].split("\n")]
    available_ingredients.sort()
    return fresh_ingerdient_ranges, available_ingredients
fir, ai = preprocess(example)
fir, ai

([[3, 5], [10, 14], [12, 18], [16, 20]], [1, 5, 8, 11, 17, 32])

In [5]:
def reduce_ranges(fresh_ranges):
    logger.info(f'Original length {len(fresh_ranges)}')
    new_list_ranges = [fresh_ranges[0]]
    for current_range in fresh_ranges[1:]:
        logger.debug(f'Evaluating range {current_range}')
        r_beg, r_end = current_range
        if r_beg <= new_list_ranges[-1][1]:
            logger.debug(f'\t{r_beg} is less than {new_list_ranges[-1][1]} so we will combine ranges')
            if r_end >= new_list_ranges[-1][1]:
                new_list_ranges[-1][1] = r_end
        else:
            new_list_ranges.append(current_range)
            logger.debug(f'\tNew range added {current_range}')
    logger.info(f'Final length {len(new_list_ranges)}')
    return new_list_ranges
            
example_ranges = reduce_ranges(fir)
example_ranges

2025-12-08 10:22:34.139 | INFO     | __main__:reduce_ranges:2 - Original length 4
2025-12-08 10:22:34.140 | DEBUG    | __main__:reduce_ranges:5 - Evaluating range [10, 14]
2025-12-08 10:22:34.140 | DEBUG    | __main__:reduce_ranges:13 - 	New range added [10, 14]
2025-12-08 10:22:34.140 | DEBUG    | __main__:reduce_ranges:5 - Evaluating range [12, 18]
2025-12-08 10:22:34.141 | DEBUG    | __main__:reduce_ranges:8 - 	12 is less than 14 so we will combine ranges
2025-12-08 10:22:34.141 | DEBUG    | __main__:reduce_ranges:5 - Evaluating range [16, 20]
2025-12-08 10:22:34.141 | DEBUG    | __main__:reduce_ranges:8 - 	16 is less than 18 so we will combine ranges
2025-12-08 10:22:34.141 | INFO     | __main__:reduce_ranges:14 - Final length 2


[[3, 5], [10, 20]]

In [6]:
def get_fresh_available_ingredients(problem_ranges, available_ingredients):
    logger.debug(f'Available ingredients {available_ingredients}')
    total_fresh_av = []
    begginings = [b for b, e in problem_ranges]
    ends = [e for b, e in problem_ranges]
    for av_ing in tqdm(available_ingredients, desc = 'Available ingredient:', total = len(available_ingredients)):
        logger.debug(f'Checking ingredient: {av_ing:.2E}')
        if av_ing < begginings[0] or av_ing > ends[-1]:
            logger.debug(f'\t{av_ing:.2E} Outside ranges ({begginings[0]:.2E} - {ends[-1]:.2E})')
            continue # if outside of ranges, not fresh
        elif av_ing in begginings or av_ing in ends:
            logger.debug(f'\t{av_ing:.2E} is beg or end')
            total_fresh_av.append(av_ing)
            continue # if any of ranges borders, then it's fresh
        else:
            for i, e in enumerate(ends):
                if av_ing <= e:
                    b = begginings[i]
                    if av_ing >= b:
                        total_fresh_av.append(av_ing)
                        logger.debug(f'\t{av_ing:.2E} is fresh because is in range {b:.2E} - {e:.2E}')
                        break
                    else:
                        logger.debug(f'\t{av_ing:.2E} is NOT fresh because is lower than range {b:.2E} - {e:.2E}')
                    
    return total_fresh_av

def solve_problem(data):
    start = time()
    fresh_ingerdient_ranges, available_ingredients = preprocess(data)
    problem_ranges = reduce_ranges(fresh_ingerdient_ranges)
    total_fresh_av = get_fresh_available_ingredients(problem_ranges, available_ingredients)
    end = time()
    logger.info(f'Part 1 took: {(end-start)*1000:.2f}ms')
    logger.info(f'Result is {len(total_fresh_av)}')
    return total_fresh_av

In [7]:
solve_problem(example)

2025-12-08 10:22:34.151 | INFO     | __main__:reduce_ranges:2 - Original length 4
2025-12-08 10:22:34.151 | DEBUG    | __main__:reduce_ranges:5 - Evaluating range [10, 14]
2025-12-08 10:22:34.151 | DEBUG    | __main__:reduce_ranges:13 - 	New range added [10, 14]
2025-12-08 10:22:34.152 | DEBUG    | __main__:reduce_ranges:5 - Evaluating range [12, 18]
2025-12-08 10:22:34.152 | DEBUG    | __main__:reduce_ranges:8 - 	12 is less than 14 so we will combine ranges
2025-12-08 10:22:34.152 | DEBUG    | __main__:reduce_ranges:5 - Evaluating range [16, 20]
2025-12-08 10:22:34.152 | DEBUG    | __main__:reduce_ranges:8 - 	16 is less than 18 so we will combine ranges
2025-12-08 10:22:34.152 | INFO     | __main__:reduce_ranges:14 - Final length 2
2025-12-08 10:22:34.152 | DEBUG    | __main__:get_fresh_available_ingredients:2 - Available ingredients [1, 5, 8, 11, 17, 32]
Available ingredient::   0%|          | 0/6 [00:00<?, ?it/s]2025-12-08 10:22:34.176 | DEBUG    | __main__:get_fresh_available_ingre

[5, 11, 17]

In [8]:
logger.remove()
logger.add(sys.stderr, level="INFO")
total_fresh_av = solve_problem(raw_data)

2025-12-08 10:22:34.189 | INFO     | __main__:reduce_ranges:2 - Original length 189
2025-12-08 10:22:34.190 | INFO     | __main__:reduce_ranges:14 - Final length 91
Available ingredient:: 100%|██████████| 1000/1000 [00:00<00:00, 109890.59it/s]
2025-12-08 10:22:34.200 | INFO     | __main__:solve_problem:34 - Part 1 took: 11.54ms
2025-12-08 10:22:34.200 | INFO     | __main__:solve_problem:35 - Result is 840


## Part 2

In [9]:
def count_fresh_ids(ranges):
    total_fresh = 0
    for rng in ranges:
        n_fresh_range = rng[1]-rng[0]+1
        total_fresh += n_fresh_range
    return total_fresh

In [10]:
def solve_problem_2(data):
    start = time()
    fresh_ingerdient_ranges, _ = preprocess(data)
    problem_ranges = reduce_ranges(fresh_ingerdient_ranges)
    total_fresh = count_fresh_ids(problem_ranges)
    start = time()
    end = time()
    logger.info(f'Part 2 took: {(end-start)*1000:.2f}ms')
    logger.info(f'Result is {total_fresh}')
    return total_fresh

solve_problem_2(example)

2025-12-08 10:22:34.210 | INFO     | __main__:reduce_ranges:2 - Original length 4
2025-12-08 10:22:34.211 | INFO     | __main__:reduce_ranges:14 - Final length 2
2025-12-08 10:22:34.211 | INFO     | __main__:solve_problem_2:8 - Part 2 took: 0.00ms
2025-12-08 10:22:34.211 | INFO     | __main__:solve_problem_2:9 - Result is 14


14

In [11]:
solve_problem_2(raw_data)

2025-12-08 10:22:34.216 | INFO     | __main__:reduce_ranges:2 - Original length 189
2025-12-08 10:22:34.216 | INFO     | __main__:reduce_ranges:14 - Final length 91
2025-12-08 10:22:34.217 | INFO     | __main__:solve_problem_2:8 - Part 2 took: 0.00ms
2025-12-08 10:22:34.217 | INFO     | __main__:solve_problem_2:9 - Result is 359913027576322


359913027576322